### Siamese Neural Networks are extremely popular for image classification,facial identification tasks,etc. 
In this notebook, I will be using a Siamese Neural network to find out whether they are suitable to distinguish between
real and forged signatures, I will link the dataset I am using in the readme.

In [12]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn 
import os
import torch.nn.functional as F
import torch.utils.data as utils
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import torchvision
from torch.autograd import Variable
from PIL import Image

device = "cuda:0" if torch.cuda.is_available() else "cpu"

Creating the dataset from the folder

In [13]:
from email.mime import image


class Dataset():
    
    def __init__(self,train_csv,train_dir,transform=None) -> None:
        self.train_df=pd.read_csv(train_csv)
        self.train_df.columns =["image1","image2","label"]
        self.train_dir = train_dir   
        self.transform = transform
    
    def __getitem__(self,index):
        image1_dir = os.path.join(self.train_df,self.train_df.iat[index,0])
        image2_dir = os.path.join(self.train_df,self.train_df.iat[index,1])
        image_0 = Image.open(image1_dir)
        image_1 = Image.open(image2_dir)
        image_0 = image_0.convert("L")
        image_1 = image_1.convert("L")
        if self.transform is not None:
            image_0 = self.transform(image_0)
            image_1 = self.transform(image_1)
        return image_0,image_1,torch.from_numpy(np.array([int(self.train_df.iat[index,2])],dtype=np.float32))
    
    def __len__(self):
        return len(self.train_df)  

In [14]:
train_csv = "./data/train_data.csv"
train_dir = "./data/train/"
dataset = Dataset(
    train_csv,
    train_dir=train_dir,
    transform=transforms.Compose(
        [transforms.Resize((105, 105)), transforms.ToTensor()]
    ),
)

In [ ]:
class SiameseNn(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11,stride=1),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),
           
            nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),

            nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
           
            nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(30976, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.5),
           
            nn.Linear(1024, 128),
            nn.ReLU(inplace=True),
           
            nn.Linear(128,2))
    
    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2